
## we won't see see the outliers, distribution of data population or any question asked in the given with the project notebook since they were answered already 

outline:

* What I did 
* What works (single user value entry)
* what doesn't work (whatever JSON we are doing and AWS deployment)
* planning for the next project


What to do next time:
- quick data exploration
- pipeline contruction and test with Nbayes.(super no hyperparameters)
- RandomSearch HyperParameter tunning (test for crash)
- quick predict using a single val from data
- flask deployment on localhost
- flask deployment on AWS
- test and see if the ML make sense or is biased(e.g. if the user is from a rural location is automatically denied)
- ask for relevant questions like ( why do we use classes for custom pipelines? or how can I modify an sklearn method that doesn't have a transform method)
- deeper data exploration
- feature engineering as part of the pipeline
- deeper development of the pipeline with custom functions
- deeper customization of flask
- more test
- repeat (agile, kanban, or whatever is your favorite development pattern)


How not to spend project time:
- answer exploratory questions and spend time producing beautiful graphs
- as a result of the exploratory questions a large portion of the code has been done manually
- this leads to a terrible pipeline design
- realize its 2100 and you are on your own
- scour stackoverflow and lose sanity
- ???
- works localy, but the port/config on AWS doesn't 
- ???
- its 1200 there's all the mentors are busy since everyone needs them to complete their project
- presentation is at 1500



In [1]:
import pandas as pd
import numpy as np

# cool custom functions
import sys
sys.path.insert(0, '/home/henri/Documents/Lighthouse-lab/lighthouse-data-notes')
import ccf_module as ccf

import matplotlib.pyplot as plt
plt.style.use('dark_background')
plt.rcParams["figure.figsize"] = (12,8)

In [46]:
df = pd.read_csv("/home/henri/Documents/Lighthouse-lab/Databases/w7-d4-db/data.csv")
df = df.drop('Loan_ID',axis=1) 

In [47]:
df.head(10)

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
5,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y
6,Male,Yes,0,Not Graduate,No,2333,1516.0,95.0,360.0,1.0,Urban,Y
7,Male,Yes,3+,Graduate,No,3036,2504.0,158.0,360.0,0.0,Semiurban,N
8,Male,Yes,2,Graduate,No,4006,1526.0,168.0,360.0,1.0,Urban,Y
9,Male,Yes,1,Graduate,No,12841,10968.0,349.0,360.0,1.0,Semiurban,N


In [4]:
labelsRes = df['Loan_Status']
df  = df.drop('Loan_Status',axis=1)

In [5]:
from sklearn.preprocessing import FunctionTransformer

In [6]:
def numFeat(data):
    return data[num_feats]

def catFeat(data):
    return data[cat_feats]

cat_feats = df.dtypes[df.dtypes == 'object'].index.tolist()
num_feats = df.dtypes[~df.dtypes.index.isin(cat_feats)].index.tolist()
keep_num = FunctionTransformer(numFeat)
keep_cat = FunctionTransformer(catFeat)

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, labelsRes,test_size=0.33, random_state=42)

In [8]:
from sklearn.experimental import enable_iterative_imputer

from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer

Outlier detection as part of a pipeline? I just don't know how to do it. There is a fit function but no transforms meaning it can't be used as part of a pipeline. The solution is to create a custom class with the sklern? WHY DID I WASTE SO MUCH TIME WITH DATA EXPLORATION AND ANSWERING THE DAMN QUESTIONS. THAT STUFF TAKES HALF A BRAIN CELL TO DO MANUALLY BUT WE WANT TO HAVE EVERYTHING DONE IN A PIPELINE WHICH TAKES A GALAXY BRAIN SINCE ITS MY SECOND DAY TOUCHING PIPELINES!!!! THE MENTORS ARE OFF LINE AT 2100 fuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuu

In [9]:
# from sklearn.ensemble import IsolationForest
# from imblearn import FunctionSampler
# def outlier_rejection(X, y):
#     """This will be our function used to resample our dataset."""
#     model = IsolationForest(max_samples=100,
#                             contamination=0.4,
#                             random_state=rng)
#     model.fit(X)
#     y_pred = model.predict(X)
#     return X[y_pred == 1], y[y_pred == 1]

In [10]:
# this jupyter lab differs greatly from my initial notebook
# 0 feature enginerring 
# 0 outlier handling
# 0 mean/mode fillna other than through MICE

numeric_transformer = Pipeline(steps=[
    ('MICE', IterativeImputer(initial_strategy='median')),
    ('scaler', StandardScaler())])
numeric_transformer.steps.append(('kbest', SelectKBest(k=3)))
# numeric_transformer.steps.append(('isoForest', FunctionSampler(func=outlier_rejection)))

In [11]:
#simple Imputer is not really needed since there are nill NaNs
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing'))])
# dummy cariable creation
categorical_transformer.steps.append(['ohc',OneHotEncoder(handle_unknown='ignore')])
# Dense Transformer needed adter one hot encoder
categorical_transformer.steps.append(['to_dense',ccf.ToDenseTransformer()])
# we only want the PCAs with the most variance
categorical_transformer.steps.append(['pca',PCA(n_components=3)])

In [12]:
preprocessor = ColumnTransformer(transformers=[
        ('num', numeric_transformer, num_feats),
        ('cat', categorical_transformer, cat_feats)])

In [13]:
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier,  RandomForestClassifier

In [14]:
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
regressors = [SVC(), GradientBoostingClassifier(), RandomForestClassifier()]

In [15]:
# sorted(base_model2.get_params().keys())
params = [{'regressors':[SVC()],
'regressors__C': [0.1, 1, 10, 100, 1000],
'regressors__gamma': [1, 0.1, 0.01, 0.001, 0.0001],
'regressors__kernel': ['poly','rbf'],

'preprocessor__cat__pca__n_components':range(3,5,1),
'preprocessor__num__kbest__k':range(2,5,1)},
    
{'regressors':[RandomForestClassifier()],
'regressors__n_estimators' : [int(x) for x in np.linspace(start = 200, stop = 1000, num = 5)],
'regressors__max_depth' : [int(x) for x in np.linspace(10, 60, num = 6)],
'regressors__min_samples_leaf' : [1, 2, 4],
'regressors__min_samples_split' : [2, 5, 10],

'preprocessor__cat__pca__n_components':range(3,5,1),
'preprocessor__num__kbest__k':range(2,5,1)},

{'regressors':[GradientBoostingClassifier()],
'regressors__max_depth':range(5,16,2), 
'regressors__min_samples_split':range(200,1001,200),
'regressors__learning_rate':[0.15,0.1,0.05,0.01,0.005,0.001],

'preprocessor__cat__pca__n_components':range(3,5,1),
'preprocessor__num__kbest__k':range(2,5,1)}
]

In [16]:
base_model2 = Pipeline([('preprocessor', preprocessor),
                      ('regressors', SVC())])
tuned_model = RandomizedSearchCV(base_model2,params,verbose=1,n_jobs=-1,n_iter=150, cv=5,random_state=42).fit(X_train, y_train)

Fitting 5 folds for each of 150 candidates, totalling 750 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:    8.2s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:   18.4s
[Parallel(n_jobs=-1)]: Done 750 out of 750 | elapsed:   32.4s finished


In [17]:
print("model score: %.3f" % tuned_model.score(X_train, y_train)) 
print("model score: %.3f" % tuned_model.score(X_test, y_test))
print(tuned_model.best_estimator_.get_params)
tuned_model.best_params_

model score: 0.849
model score: 0.798
<bound method Pipeline.get_params of Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('MICE',
                                                                   IterativeImputer(initial_strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler()),
                                                                  ('kbest',
                                                                   SelectKBest(k=4))]),
                                                  ['ApplicantIncome',
                                                   'CoapplicantIncome',
                                                   'LoanAmount',
                                                   'Loan_Amount_Term',
                                          

{'regressors__n_estimators': 1000,
 'regressors__min_samples_split': 10,
 'regressors__min_samples_leaf': 4,
 'regressors__max_depth': 50,
 'regressors': RandomForestClassifier(max_depth=50, min_samples_leaf=4, min_samples_split=10,
                        n_estimators=1000),
 'preprocessor__num__kbest__k': 4,
 'preprocessor__cat__pca__n_components': 3}

In [18]:
import pickle
# save the model to disk
filename = 'pickledRandCVModel.sav' # .sav file extension are commonly used in video games as savepoint
pickle.dump(tuned_model, open(filename, 'wb'))

In [19]:
test2 = ['Male', 'Yes', '0', 'Graduate', 'Yes', 10000, 10000, 10, 10, 1, 'Urban']

In [22]:
tuned_model.predict(X_test)

array(['Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'N', 'N', 'N', 'Y', 'Y',
       'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'N', 'Y', 'N', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'N',
       'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y',
       'N', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'N', 'Y', 'N', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y',
       'N', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y',
       'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N',
       'Y', 'N', 'N', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y

In [40]:
X_test.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
350,Male,Yes,0,Graduate,No,9083,0.0,228.0,360.0,1.0,Semiurban
377,Male,Yes,0,Graduate,No,4310,0.0,130.0,360.0,NaN,Semiurban
163,Male,Yes,2,Graduate,No,4167,1447.0,158.0,360.0,1.0,Rural
609,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural
132,Male,No,0,Graduate,No,2718,0.0,70.0,360.0,1.0,Semiurban


In [43]:
 ds = pd.Series(test2,index=['Gender','Married','Dependents','Education','Self_Employed','ApplicantIncome','CoapplicantIncome','LoanAmount','Loan_Amount_Term','Credit_History','Property_Area',])
 ds.to_frame().T

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,Male,Yes,0,Graduate,Yes,10000,10000,10,10,1,Urban


In [44]:
tuned_model.predict( ds.to_frame().T)

array(['Y'], dtype=object)

In [ ]:
#transferring files from localhost to aws instance
#sudo scp -i "/home/henri/Documents/Lighthouse-lab/keys/lighthouse-lab-ec2-key.pem" '/home/henri/Documents/Lighthouse-lab/lighthouse-data-notes/ccf_module.py' ec2-user@ec2-99-79-72-222.ca-central-1.compute.amazonaws.com:~/deployedapp/
